#2do Parcial Inteligencia Artificial 1 (SIS420)

Univ. Lino Fernando Villca Jaita (Ing. en Ciencias De La Computación)

In [9]:
import numpy as np

class Board():
    def __init__(self):
        self.state = np.zeros((4, 4))  # Inicialización del tablero como una matriz de ceros 4x4

    def valid_moves(self):
        # Devuelve una lista de todas las posiciones vacías en el tablero
        return [(i, j) for j in range(4) for i in range(4) if self.state[i, j] == 0]

    def update(self, symbol, row, col):
        # Actualiza el tablero con el símbolo del jugador en la posición dada
        if self.state[row, col] == 0:
            self.state[row, col] = symbol
        else:
            raise ValueError("Movimiento ilegal !")

    def is_game_over(self):
        # Comprueba si hay un ganador o empate y devuelve el resultado
        if (self.state.sum(axis=0) == 4).sum() >= 1 or (self.state.sum(axis=1) == 4).sum() >= 1:
            return 1
        if (self.state.sum(axis=0) == -4).sum() >= 1 or (self.state.sum(axis=1) == -4).sum() >= 1:
            return -1
        diag_sums = [
            sum([self.state[i, i] for i in range(4)]),
            sum([self.state[i, 4 - i - 1] for i in range(4)]),
        ]
        if diag_sums[0] == 4 or diag_sums[1] == 4:
            return 1
        if diag_sums[0] == -4 or diag_sums[1] == -4:
            return -1
        if len(self.valid_moves()) == 0:
            return 0
        return None  # Devuelve None si el juego sigue en curso

    def reset(self):
        self.state = np.zeros((4, 4))  # Reinicia el tablero al estado inicial

In [10]:
from tqdm import tqdm

class Game():
    def __init__(self, player1, player2):
        player1.symbol = 1  # Asigna el símbolo 1 al player 1
        player2.symbol = -1  # Asigna el símbolo -1 al player 2
        self.players = [player1, player2]  # Lista de jugadores
        self.board = Board()  # Inicializa el tablero

    def selfplay(self, rounds=100):
        wins = [0, 0]  # Contador de victorias para cada jugador
        for i in tqdm(range(1, rounds + 1)):
            self.board.reset()  # Reinicia el tablero para una nueva partida
            for player in self.players:
                player.reset()  # Reinicia el jugador para una nueva partida
            game_over = False
            while not game_over:
                for player in self.players:
                    action = player.move(self.board)  # Obtiene el movimiento del jugador
                    self.board.update(player.symbol, action[0], action[1])  # Actualiza el tablero
                    for player in self.players:
                        player.update(self.board)  # Actualiza el estado interno de los jugadores
                    if self.board.is_game_over() is not None:
                        game_over = True
                        break
            self.reward()  # Calcula las recompensas al final de la partida
            for ix, player in enumerate(self.players):
                if self.board.is_game_over() == player.symbol:
                    wins[ix] += 1  # Incrementa el contador de victorias del jugador correspondiente
        return wins  # Devuelve el número de victorias para cada jugador

    def reward(self):
        # Calcula las recompensas al final de la partida
        winner = self.board.is_game_over()
        if winner == 0:  # Empate
            for player in self.players:
                player.reward(0.5)
        else:  # Recompensa para el jugador que gana
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)
                else:
                    player.reward(0)


In [11]:
class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.value_function = {}  # Tabla con pares estado -> valor
        self.alpha = alpha  # Learning rate
        self.positions = []  # Guarda todas las posiciones de la partida
        self.prob_exp = prob_exp  # Probabilidad de explorar

    def reset(self):
        self.positions = []  # Reinicia la lista de posiciones

    def move(self, board, explore=True):
        valid_moves = board.valid_moves()  # Obtiene los movimientos válidos en el tablero
        if explore and np.random.uniform(0, 1) < self.prob_exp:
            # Exploración: movimiento aleatorio
            ix = np.random.choice(len(valid_moves))
            return valid_moves[ix]
        # Explotación: movimiento con el mayor valor
        max_value = -1000
        for row, col in valid_moves:
            next_board = board.state.copy()
            next_board[row, col] = self.symbol
            next_state = str(next_board.reshape(4*4))  # Representación del siguiente estado
            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state)
            if value >= max_value:
                max_value = value
                best_row, best_col = row, col
        return best_row, best_col  # Devuelve la mejor posición para moverse

    def update(self, board):
        self.positions.append(str(board.state.reshape(4*4)))  # Guarda la posición actual

    def reward(self, reward):
        # Actualiza la función de valor al final de la partida
        for p in reversed(self.positions):
            if self.value_function.get(p) is None:
                self.value_function[p] = 0
            self.value_function[p] += self.alpha * (reward - self.value_function[p])
            reward = self.value_function[p]

In [13]:
agent1 = Agent(prob_exp=0.5)  # Agente 1
agent2 = Agent()  # Agente 2

game = Game(agent1, agent2)  # Juego entre los dos agentes

game.selfplay(10000)  # Jugar varias partidas

100%|██████████| 10000/10000 [04:01<00:00, 41.46it/s]


[3101, 2595]

In [14]:
import pandas as pd
# Ordenar la tabla de función de valor por valor descendente
funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)
# Crear un DataFrame con los estados y sus valores
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})

tabla  # Mostrar la tabla de función de valor

,estado,valor
0,[ 0. 1. 0. -1. 0. 1. 0. 0. 0. 1. 0. -...,0.999023
1,[ 0. 1. 0. 0. 0. 1. 0. -1. 0. 1. 0. -...,0.984375
2,[ 0. 1. -1. 0. 0. 1. 0. 0. 0. 1. 0. -...,0.968750
3,[ 0. 0. 0. 1. 0. 0. 1. -1. 0. 1. 0. -...,0.968750
4,[ 1. 0. -1. 1. 1. 1. -1. -1. 1. -1. -1. -...,0.937500
...,...,...
89396,[ 1. 0. 0. 1. -1. 0. 0. -1. 0. 1. 0. -...,0.000000
89397,[ 1. 0. 0. 1. 0. 0. 0. -1. 0. 1. 0. -...,0.000000
89398,[ 1. 0. 0. 0. 0. 0. 0. -1. 0. 1. 0. -...,0.000000
89399,[ 1. 1. 0. -1. 0. 0. 0. -1. 1. 1. 0. -...,0.000000


In [15]:
import pickle

# Guardar la función de valor del agente en un archivo pickle
with open('agente.pickle', 'wb') as handle:
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)
